# Fast & Cheap
Le projet d'analyse de données et de régression linéaire afin d'aider *Martin* à estimer sa future voiture en fonction des caractéristiques souhaitées :
- Véhicule moins de 7 ans
- Pas plus de 100 000km au compteur
- En boîte manuelle

## Récupération des données et nettoyage du dataset

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [13]:
# Chargement des données et analyses générales
df = pd.read_csv('data/raw/carData.csv', encoding='latin-1')
print("5 premières lignes du dataset")
print(df.head())
print('----------------------------------------------------------------')
print("Analyse descriptive des variables numérique")
print(df.describe())
print('----------------------------------------------------------------')
print("Infos générales (types de variables, nombre des lignes etc...)")
print(df.info())


5 premières lignes du dataset
  Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
0     ritz  2014           3.35           5.59       27000    Petrol   
1      sx4  2013           4.75           9.54       43000    Diesel   
2     ciaz  2017           7.25           9.85        6900    Petrol   
3  wagon r  2011           2.85           4.15        5200    Petrol   
4    swift  2014           4.60           6.87       42450    Diesel   

  Seller_Type Transmission  Owner  
0      Dealer       Manual      0  
1      Dealer       Manual      0  
2      Dealer       Manual      0  
3      Dealer       Manual      0  
4      Dealer       Manual      0  
----------------------------------------------------------------
Analyse descriptive des variables numérique
              Year  Selling_Price  Present_Price     Kms_Driven       Owner
count   301.000000     301.000000     301.000000     301.000000  301.000000
mean   2013.627907       4.661296       7.628472   36947.205

## Nettoyage du dataset

In [16]:
# Suppresion des lignes doublés
df = df.drop_duplicates()

# Nombre de lignes doublées supprimées
print('\nAprès suppression lignes doublées :')
print(df.info())

# Vérification de valeurs manquantes
print("\nVérification de valeurs manquantes :")
print(df.isnull().sum())

print("\nComme il n'y a pas de valeurs manquantes, aucune suppression de lignes")


Après suppression lignes doublées :
<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       299 non-null    object 
 1   Year           299 non-null    int64  
 2   Selling_Price  299 non-null    float64
 3   Present_Price  299 non-null    float64
 4   Kms_Driven     299 non-null    int64  
 5   Fuel_Type      299 non-null    object 
 6   Seller_Type    299 non-null    object 
 7   Transmission   299 non-null    object 
 8   Owner          299 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 23.4+ KB
None

Vérification de valeurs manquantes :
Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

Comme il n'y a pas de valeurs manquantes, aucune suppression de lignes


## Sauvegarde du dataset propre

In [17]:
# Sauvegarder le tableau nettoyé dans data/clean
os.makedirs('data/clean', exist_ok=True)

# Copier le DataFrame actuel (après nettoyage/transformations)
df_clean = df.copy()

# Chemin de sortie
path_wide = 'data/clean/carData_clean.csv'

# Enregistrer les fichiers
try:
    df_clean.to_csv(path_wide, index=False)
    print(f"Sauvegardé : {path_wide} (shape={df_clean.shape})")
except Exception as e:
    print('Erreur lors de la sauvegarde :', e)

Sauvegardé : data/clean/carData_clean.csv (shape=(299, 9))
